<a href="https://colab.research.google.com/github/jsimpauco/dsc180a-q1-project/blob/main/DSC_180A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing required packages #
!pip install datasets
!pip install biopython
!pip install pysam
!pip install pyzstd
!pip install zstandard

# Importing packages #
import torch
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import pysam
import requests
from Bio.Seq import Seq
from datasets import load_dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 82.6 MB/s eta 0:00:00
   ━━━━━

In [ ]:
# URL with specified split #
url = "https://datasets-server.huggingface.co/rows?dataset=songlab%2Fgenomes-brassicales-balanced-v1&config=default&split=train"
response = requests.get(url)

# Check if the request was successful #
data = response.json()
df = pd.DataFrame(data["rows"])
df_expanded = pd.json_normalize(df['row'])
df_expanded.shape


(100, 6)

In [ ]:
# Loading dataset #
ds = load_dataset("songlab/genomes-brassicales-balanced-v1")

DatasetDict({
    train: Dataset({
        features: ['assembly', 'chrom', 'start', 'end', 'strand', 'seq'],
        num_rows: 4066336
    })
    validation: Dataset({
        features: ['assembly', 'chrom', 'start', 'end', 'strand', 'seq'],
        num_rows: 127748
    })
    test: Dataset({
        features: ['assembly', 'chrom', 'start', 'end', 'strand', 'seq'],
        num_rows: 186636
    })
})

In [ ]:
# Converting dataset to pandas DataFrame #
df = ds['train'].to_pandas()
df

,assembly,chrom,start,end,strand,seq
0,GCF_000309985.2,NC_024803.2,44350126,44350638,-,AACGTTACACAAACCCAAACAAGAAAATACATCAAAAGTCACCCTA...
1,GCF_000801105.1,NW_017353143.1,6209626,6210138,+,CATCTTGAGCTTCCTCGAAAACAATCTGCTCTGTCTCTACATCATT...
2,GCF_000801105.1,NW_017353146.1,2729759,2730271,-,tacagataCAACCATCCTTGGAATGTATCATTCCACATTATACCAT...
3,GCF_000463585.1,NW_010966372.1,1523140,1523652,+,Agggaaaaatatatgaaaatttctgaatttgtaatttggtataata...
4,GCF_000001735.4,NC_003071.7,18874538,18875050,+,GGAATGTACTAATAAACTAGAAAAGATCTTGGGTCTGACTAATGCA...
...,...,...,...,...,...,...
4066331,GCF_000478725.1,NW_006256909.1,5629503,5630015,-,AAATTgttcaaatttatataaacATAATCATCATCGTGAAGAACAT...
4066332,GCF_000150535.2,NW_019014633.1,827130,827642,+,ttaaaaaaaagtactCAGGAGGGTAATTTTATCTGCTATTGATGTA...
4066333,GCF_000463585.1,NW_010966177.1,3744061,3744573,+,TCAGCTTCGGAGACAAAGGTCAGTCGGATCAGCAAGAAACACCATT...
4066334,GCF_000633955.1,NC_025690.1,13142119,13142631,-,CCAAACTCGCATCTCATCCATgccaaaaattatttatattatagtt...


In [ ]:
# Filtering by chromosome #
filtered_data = df[df['chrom'] == 'NC_003071.7']
filtered_data

,assembly,chrom,start,end,strand,seq
4,GCF_000001735.4,NC_003071.7,18874538,18875050,+,GGAATGTACTAATAAACTAGAAAAGATCTTGGGTCTGACTAATGCA...
24,GCF_000001735.4,NC_003071.7,11719046,11719558,-,ATAATCATGGAGCCATATACAATTTAAGATGGTGCAAAAATGGCGA...
34,GCF_000001735.4,NC_003071.7,6908827,6909339,-,TTTCTGGTAAATCCGATTTTGGGAAACTCTAATTTGATCTCGAGAT...
68,GCF_000001735.4,NC_003071.7,15915806,15916318,+,TGTACTTCTCCCTTTCCTTGGGGTCATTGCATATAACCTCTTTCAC...
159,GCF_000001735.4,NC_003071.7,11195806,11196318,+,tttAGTGGctagaaagaaaacttttaacgATAAATTTACTTatgca...
...,...,...,...,...,...,...
4066042,GCF_000001735.4,NC_003071.7,5217771,5218283,+,AGGAACCTCTTTTATTCCTCTTTGAAAATTATTGTGCTAAATTTAC...
4066099,GCF_000001735.4,NC_003071.7,4493950,4494462,+,GGAAGTGATTGTAAGTGATTGTTTTTGCGAAAAATGATAATGAGTA...
4066259,GCF_000001735.4,NC_003071.7,16128521,16129033,+,CATGATTGTGGCCGGTCCAATCACAGCGAACGCGCTTATGAGTTGT...
4066270,GCF_000001735.4,NC_003071.7,1626166,1626678,+,GCCTCACAGAGAGTTATAACACTATATCTATTGTCAGTAACTTGTA...
